In [1]:
import numpy as np
import scipy
import sympy as sp
import matplotlib.pyplot as plt

In [2]:
s = sp.symbols('s')
r = s - 1

rho = sp.symbols('rho')
rho1 = rho / (1-2**(-r))

s1 = 2*s-1
M1 = 2**s1 / (1-2**(-s1)) * rho1

s2 = 2*s-2
M2 = (2**s2 * (2*rho1+rho1**2)) / (1-2**(-s2) - 2**s2 * (2*rho1+rho1**2))

M3 = (2**s * rho1) / ((1-2**(-s)) * (1-rho1) - 2**s * rho1)

gamma = sp.symbols('gamma')
Linv_norm = gamma**2 * (1+M1) * (1+M2) * (1+M3)

K_r = (3-2**(-r)) * 2**r / (1-2**(-r))**2
K_CM_rs = 2 / (1-2**(-r)) / (1-2**(-s)) / (1-2**(-s-r)) * 2**(r+2*s) \
           + (3+2**(-r)) / (1-2**(-r))**2 * 2**(2*r+s)

R_1 = K_CM_rs * (2*K_r*rho+8) / (1-M1) * rho**3 / (1 - K_r*rho)

# paralinearization residual
eps = sp.symbols('epsilon')
R_PL = 1000 * eps**2

# full right-hand side term for the fixed-point formulation
RHS = Linv_norm * (R_1 + R_PL)
RHS

gamma**2*(1000*epsilon**2 + rho**3*(2**(3*s - 2)*(2**(1 - s) + 3)/(1 - 2**(1 - s))**2 + 2*2**(3*s - 1)/((1 - 1/2**s)*(1 - 2**(1 - 2*s))*(1 - 2**(1 - s))))*(2*2**(s - 1)*rho*(3 - 2**(1 - s))/(1 - 2**(1 - s))**2 + 8)/((-2**(s - 1)*rho*(3 - 2**(1 - s))/(1 - 2**(1 - s))**2 + 1)*(-2**(2*s - 1)*rho/((1 - 2**(1 - 2*s))*(1 - 2**(1 - s))) + 1)))*(2**(2*s - 2)*(rho**2/(1 - 2**(1 - s))**2 + 2*rho/(1 - 2**(1 - s)))/(-2**(2 - 2*s) - 2**(2*s - 2)*(rho**2/(1 - 2**(1 - s))**2 + 2*rho/(1 - 2**(1 - s))) + 1) + 1)*(2**s*rho/((1 - 2**(1 - s))*(-2**s*rho/(1 - 2**(1 - s)) + (1 - 1/2**s)*(-rho/(1 - 2**(1 - s)) + 1))) + 1)*(2**(2*s - 1)*rho/((1 - 2**(1 - 2*s))*(1 - 2**(1 - s))) + 1)

In [3]:
# fix s, gamma
RHS_sim = sp.simplify(RHS.subs(s, 3.1).subs(gamma, (3+np.sqrt(5))/8))
RHS_sim

# num, den = sp.fraction(RHS_s)
# sp.Poly(num, gamma, eps).as_expr()

0.405073395048388*(0.883370876057899 - 1.15211004698814*rho)*(49.2816685068612*rho + 1)*(1000*epsilon**2*(20.1759219394273*rho - 1)*(49.2816685068612*rho - 1) + rho**3*(73576.0050179016*rho + 14586.8932758153))/((12.334737935231*rho - 0.883370876057899)*(20.1759219394273*rho - 1)*(49.2816685068612*rho - 1)*(31.2627916217269*rho**2 + 47.940975248148*rho - 0.945590589793992))

In [15]:
from ipywidgets import interact, FloatSlider, Layout

RHS_sim_func = sp.lambdify([eps, rho], RHS_sim, 'numpy')

@interact(eps_val=FloatSlider(value=0.001, min=0, max=0.01, step=0.00001, 
                        description='ε:', continuous_update=True,
                        style={'description_width': 'initial'},
                        layout=Layout(width='80%'), readout_format='.4f'))
def plot_interactive(eps_val):
    rho_max = 0.006
    rho_vals = np.linspace(0, rho_max, 10001)
    y_vals = RHS_sim_func(eps_val, rho_vals)
    
    plt.figure(figsize=(8, 5))
    plt.plot(rho_vals, rho_vals, 'k-', lw=2, label='ρ')
    plt.plot(rho_vals, y_vals, 'b-', lw=2, label=f'ε={eps_val:.4f}')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()


# plt.figure(figsize=(8,12))
# plt.plot(rho_vals, rho_vals, 'k-', label='rho')
# for eps_val in eps_list:
#     res_vals = RHS_sim_func(eps_val, rho_vals)
#     plt.plot(rho_vals, res_vals, label=f'eps={eps_val}')
# plt.xlabel('rho')
# plt.legend()
# plt.show()

interactive(children=(FloatSlider(value=0.001, description='ε:', layout=Layout(width='80%'), max=0.01, readout…

In [6]:
# Previous codes

M_1 = lambda s,rho: 2**s / (1-2**(-s)) * rho
M_2 = lambda s,rho: (2**s * (2*rho+rho**2)) / (1-2**(-s) - 2**s * (2*rho+rho**2))
M_3 = lambda s,rho: (2**s * rho) / ((1-2**(-s)) * (1-rho) - 2**s * rho)

rho1 = lambda r,rho: rho / (1-2**(-r))

Linv_norm = lambda gamma,s,rho: gamma**2 * (1+M_1(2*s-1,rho1(s-1,rho))) \
                                * (1+M_2(2*s-2,rho1(s-1,rho))) * (1+M_3(s,rho1(s-1,rho)))

K_r = lambda r: (3-2**(-r)) * 2**r / (1-2**(-r))**2

def K_CM_rs(r, s):
    K = 2 / (1.0-2**(-r)) / (1.0-2**(-s)) / (1.0-2**(-s-r)) * 2**(r+2*s)
    K += (3+2**(-r)) / (1-2**(-r))**2 * 2**(2*r+s)
    return K

def R_1(s, rho):
    R = K_CM_rs(s-1, s) * (2*K_r(s-1)*rho+8) / (1-M_1(2*s-1,rho))
    R = R * rho**3 / (1 - K_r(s-1)*rho)
    return R

gamma = (3+np.sqrt(5))/8
s = 3.1
eps_list = [0.001, 0.002, 0.0025, 0.0026]
rho_max = 0.006
rho_vals = np.linspace(0, rho_max, 10001)

plt.figure(figsize=(8,12))
plt.plot(rho_vals, rho_vals, 'k-', label='rho')
for eps in eps_list:
    res_vals = Linv_norm(gamma, s, rho_vals) * (R_1(s, rho_vals) + 1000*eps**2)
    plt.plot(rho_vals, res_vals, label=f'eps={eps}')
plt.xlabel('rho')
plt.legend()
plt.show()

array([0.00289586, 0.00289605, 0.00289625, ..., 0.01038225, 0.01038466,
       0.01038707])